# Case 1 Part 2: Anomaly Detection

This notebook showcases the 2nd part of the analytics use case to be tackled in a real-time alerting system:

`Hourly consumption for a household is higher than 1 standard deviation of that household's historical mean consumption for that hour.`

The second part here is to read the data stream from both `readings_prepared` and `alert_1_stats` and use them to detect data anomalies -- the ones that go above 1 standard deviation from the mean. The detected anomalies are then stored in a persistent data store, which in this case is Google BigQuery.

BigQuery is chosen for the fit of further analytical queries down the line. We might be interested to do some BI or advanced analysis down the line. It is also serverless -- we just need to define the Datasets and Tables.


## Setup

Import all the required libraries and set the stream configuration variables.

In [1]:
import spark.implicits._
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.streaming.Trigger
import org.apache.spark.sql.expressions.Window
import org.apache.spark.sql._

Intitializing Scala interpreter ...

Spark Web UI available at http://spark-alert-1-detect-m:8088/proxy/application_1583161287214_0002
SparkContext available as 'sc' (version = 2.4.5, master = yarn, app id = application_1583161287214_0002)
SparkSession available as 'spark'


import spark.implicits._
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.streaming.Trigger
import org.apache.spark.sql.expressions.Window
import org.apache.spark.sql._


In [2]:
val kafkaBootstrapServer = "kafka-m:9092"
val kafkaReadingsTopic = "readings_prepared"
val kafkaStatsTopic = "alert_1_stats"
val kafkaDedupWatermarkTime = "1 minute"
val joinWatermarkTime = "1 minute"
val bigQueryTargetTable = "smartplugs.alert_1_anomaly"
val bigQueryTempBucket = "pandora-sde-case/alert_1"
val outputTriggerTime = "1 minute"

kafkaBootstrapServer: String = kafka-m:9092
kafkaReadingsTopic: String = readings_prepared
kafkaStatsTopic: String = alert_1_stats
kafkaDedupWatermarkTime: String = 1 minute
joinWatermarkTime: String = 1 minute
bigQueryTargetTable: String = smartplugs.alert_1_anomaly
bigQueryTempBucket: String = pandora-sde-case/alert_1
outputTriggerTime: String = 1 minute


## Define The Required Schema

In [3]:
// This will be used to give the source `readings_prepared` stream data a schema
val readingsSchema = StructType(Seq(
    StructField("message_id", StringType, false),
    StructField("reading_ts", TimestampType, false),
    StructField("reading_value", FloatType, false),
    StructField("reading_type", IntegerType, false),
    StructField("plug_id", IntegerType, false),
    StructField("household_id", IntegerType, false),
    StructField("house_id", IntegerType, false)
))

val statsSchema = StructType(Seq(
    StructField("house_id", IntegerType, false),
    StructField("hour", IntegerType, false),
    StructField("mean", FloatType, false),
    StructField("m2", FloatType, false),
    StructField("variance", FloatType, false),
    StructField("std_dev", FloatType, false),
    StructField("count", LongType, false),
    StructField("last_ts", TimestampType, false)
))

readingsSchema: org.apache.spark.sql.types.StructType = StructType(StructField(message_id,StringType,false), StructField(reading_ts,TimestampType,false), StructField(reading_value,FloatType,false), StructField(reading_type,IntegerType,false), StructField(plug_id,IntegerType,false), StructField(household_id,IntegerType,false), StructField(house_id,IntegerType,false))
statsSchema: org.apache.spark.sql.types.StructType = StructType(StructField(house_id,IntegerType,false), StructField(hour,IntegerType,false), StructField(mean,FloatType,false), StructField(m2,FloatType,false), StructField(variance,FloatType,false), StructField(std_dev,FloatType,false), StructField(count,LongType,false), StructField(last_ts,TimestampType,false))


### Read and Parse The Input Data Streams

There are 2 input streams this time, `readings_prep` and `alert_1_stats` topic. They will be joined to detect anomalies

In [4]:
// Drop duplicates if seen in an arbitrary watermark. Bounds are necessary so that Spark does not store 
// ALL records in the state
val readings = spark
    .readStream 
    .format("kafka")
    .option("kafka.bootstrap.servers", kafkaBootstrapServer)
    .option("subscribe", kafkaReadingsTopic)
    .load()
    .selectExpr("CAST(value AS STRING)")
    .select(from_json($"value", readingsSchema).as("data"))
    .select($"data.*")
    .withWatermark("reading_ts", kafkaDedupWatermarkTime) 
    .dropDuplicates()
    .filter($"reading_type" === 1) // Only take the "current load" measurement

readings: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [message_id: string, reading_ts: timestamp ... 5 more fields]


In [5]:
// We don't need to deduplicate the stats, but we drop the ones with mean=0
val stats = spark
    .readStream 
    .format("kafka")
    .option("kafka.bootstrap.servers", kafkaBootstrapServer)
    .option("subscribe", kafkaStatsTopic)
    .load()
    .selectExpr("CAST(value AS STRING)")
    .select(from_json($"value", statsSchema).as("data"))
    .select($"data.house_id", $"data.hour", $"data.mean", $"data.std_dev", $"data.last_ts")
    .filter($"mean" > 0.0)


stats: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [house_id: int, hour: int ... 3 more fields]


#### Peek at The Input Data Streams

##### Readings

In [6]:
val readingsQuery = readings.writeStream.format("memory").queryName("readings").start()
Thread.sleep(10000)
readingsQuery.status

readingsQuery: org.apache.spark.sql.streaming.StreamingQuery = org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@26fe9d32
res0: org.apache.spark.sql.streaming.StreamingQueryStatus =
{
  "message" : "Processing new data",
  "isDataAvailable" : true,
  "isTriggerActive" : true
}


In [25]:

spark.sql("select * from readings").show()

+----------+-------------------+-------------+------------+-------+------------+--------+
|message_id|         reading_ts|reading_value|reading_type|plug_id|household_id|house_id|
+----------+-------------------+-------------+------------+-------+------------+--------+
| 118504114|2013-09-01 17:53:00|          0.0|           1|      0|           0|       9|
| 115998526|2013-09-01 17:27:20|          0.0|           1|      0|           0|       0|
| 116872749|2013-09-01 17:36:20|          0.0|           1|      1|           0|       9|
| 118371375|2013-09-01 17:51:40|          0.0|           1|      2|           0|       4|
| 117153098|2013-09-01 17:39:20|          0.0|           1|      0|           0|       5|
| 117306922|2013-09-01 17:41:00|       50.627|           1|      0|           0|       8|
| 118503302|2013-09-01 17:53:00|       41.056|           1|      2|           0|       0|
| 118868629|2013-09-01 17:56:40|      122.738|           1|      1|           0|       1|
| 11972589

In [27]:
// readingsQuery.stop()
readingsQuery.lastProgress

res18: org.apache.spark.sql.streaming.StreamingQueryProgress =
{
  "id" : "a82e2b35-dd47-44e8-bc23-0ef97d2e43d5",
  "runId" : "70669817-58c1-4f28-9075-4c437d0915a0",
  "name" : "readings",
  "timestamp" : "2020-03-02T15:31:02.071Z",
  "batchId" : 3,
  "numInputRows" : 0,
  "inputRowsPerSecond" : 0.0,
  "processedRowsPerSecond" : 0.0,
  "durationMs" : {
    "addBatch" : 46204,
    "getBatch" : 0,
    "getEndOffset" : 0,
    "queryPlanning" : 52,
    "setOffsetRange" : 0,
    "triggerExecution" : 46862,
    "walCommit" : 117
  },
  "eventTime" : {
    "watermark" : "2013-09-01T18:07:00.000Z"
  },
  "stateOperators" : [ {
    "numRowsTotal" : 32,
    "numRowsUpdated" : 0,
    "memoryUsedBytes" : 217999,
    "customMetrics" : {
      "loadedMapCacheHitCount" : 600,
      "loadedMapCacheMiss...

In [9]:
readingsQuery.status

res3: org.apache.spark.sql.streaming.StreamingQueryStatus =
{
  "message" : "Waiting for data to arrive",
  "isDataAvailable" : false,
  "isTriggerActive" : false
}


##### Stats

In [10]:
val statsQuery = stats.writeStream.format("memory").queryName("stats").start()
statsQuery.status

statsQuery: org.apache.spark.sql.streaming.StreamingQuery = org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@2fcfb0d7
res4: org.apache.spark.sql.streaming.StreamingQueryStatus =
{
  "message" : "Getting offsets from KafkaV2[Subscribe[alert_1_stats]]",
  "isDataAvailable" : false,
  "isTriggerActive" : true
}


In [26]:
// Thread.sleep(10000)
spark.sql("select * from stats").show()

+--------+----+------------+-----------+-------------------+
|house_id|hour|        mean|    std_dev|            last_ts|
+--------+----+------------+-----------+-------------------+
|       3|  22|0.0065128207| 0.07044692|2013-08-31 22:55:40|
|       1|   0|   3.0833957| 0.50312406|2013-09-01 00:52:20|
|       6|   1| 0.026075268|  0.1238482|2013-09-01 01:24:40|
|       3|   1| 0.028548673| 0.14973912|2013-09-01 01:13:40|
|       7|   4|   1.8362736|  6.7292614|2013-09-01 04:13:40|
|       6|  22|  0.00566346|0.057756197|2013-08-31 22:19:20|
|       9|   4|    1.582257|   6.700217|2013-09-01 04:41:40|
|       8|   3|   10.992943|   3.251793|2013-09-01 03:44:20|
|       1|   2|   2.9568825| 0.30885485|2013-09-01 02:39:40|
|       6|   0|  0.01244898|0.086696796|2013-09-01 00:10:20|
|       4|  22| 0.016184205| 0.19953239|2013-08-31 22:59:20|
|       4|   0|  0.03087729| 0.27790377|2013-09-01 00:38:00|
|       0|   5|    7.169792|   4.563983|2013-09-01 05:52:20|
|       1|  22|    4.302

In [28]:
// statsQuery.stop()
statsQuery.lastProgress

res19: org.apache.spark.sql.streaming.StreamingQueryProgress =
{
  "id" : "7b4fdc65-37cb-4bb0-9d49-cde36a2874a1",
  "runId" : "96b30d71-661f-4238-90af-9f082fe2827e",
  "name" : "stats",
  "timestamp" : "2020-03-02T15:31:42.115Z",
  "batchId" : 3,
  "numInputRows" : 0,
  "inputRowsPerSecond" : 0.0,
  "processedRowsPerSecond" : 0.0,
  "durationMs" : {
    "getEndOffset" : 1,
    "setOffsetRange" : 0,
    "triggerExecution" : 1
  },
  "stateOperators" : [ ],
  "sources" : [ {
    "description" : "KafkaV2[Subscribe[alert_1_stats]]",
    "startOffset" : {
      "alert_1_stats" : {
        "0" : 110
      }
    },
    "endOffset" : {
      "alert_1_stats" : {
        "0" : 110
      }
    },
    "numInputRows" : 0,
    "inputRowsPerSecond" : 0.0,
    "processedRowsPerSecond" : 0.0
  } ],
  "s...

In [13]:
statsQuery.status

res7: org.apache.spark.sql.streaming.StreamingQueryStatus =
{
  "message" : "Waiting for data to arrive",
  "isDataAvailable" : false,
  "isTriggerActive" : false
}


## Join Both Data Streams


We can only compare the readings with the past statistics. The `reading_ts` value of `readings_prepared` stream needs to be bigger than the `last_ts` value of the `alert_1_stats` stream. We need to ensure we are comparing the stats from the right `house_id` and `hour`.

`owever, as the stream runs, the size of streaming state will keep growing indefinitely as all past input must be saved as any new input can match with any input from the past. To avoid unbounded state, you have to define additional join conditions such that indefinitely old inputs cannot match with future inputs and therefore can be cleared from the state. In other words, you will have to do the following additional steps in the join.`

After joining, we need to ensure that we only compare the readings with their latest past statistics.

The alert_1_stats stream is assumed to be slower to arrive compared to the readings_prepared stream, since it needs to wait at least 2 records to start calculating, and the default write trigger is every 30 seconds. 


The alert_1_stats may contain multiple rows with different `last_ts` and stats values as well, and we're going to solve that with an aggregation after joining. We're going to take the max `last_ts` that is still earlier than the `reading_ts` value.




In [14]:
// Anomaly detection is done by getting the latest std_dev and mean value
// Then act on it by comparing to the 
val anomaly = readings
    .withColumnRenamed("house_id", "readings_house_id")
    .withWatermark("reading_ts", joinWatermarkTime)
    .join(
        stats.withWatermark("last_ts", joinWatermarkTime),
        // Join conditions
        $"reading_ts" > $"last_ts" &&
        hour($"reading_ts") === $"hour" &&
        $"readings_house_id" === $"house_id",
        // Join type
        "inner"
    )
    .filter($"reading_value" > $"mean" + $"std_dev")
    .drop("readings_house_id")

anomaly: org.apache.spark.sql.DataFrame = [message_id: string, reading_ts: timestamp ... 9 more fields]


In [15]:
val peekQuery = anomaly
    .writeStream
    .format("memory")
    .queryName("anomaly")
    .start()

peekQuery: org.apache.spark.sql.streaming.StreamingQuery = org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@2e27451


In [36]:
spark.sql("select * from anomaly").show()

+----------+-------------------+-------------+------------+-------+------------+--------+----+--------+----------+-------------------+
|message_id|         reading_ts|reading_value|reading_type|plug_id|household_id|house_id|hour|    mean|   std_dev|            last_ts|
+----------+-------------------+-------------+------------+-------+------------+--------+----+--------+----------+-------------------+
| 118802374|2013-09-01 17:56:00|      149.446|           1|      1|           0|       1|  17|138.2499| 10.936097|2013-09-01 17:43:20|
| 117686988|2013-09-01 17:45:00|      152.129|           1|      1|           0|       1|  17|138.2499| 10.936097|2013-09-01 17:43:20|
| 117753806|2013-09-01 17:45:40|      153.223|           1|      1|           0|       1|  17|138.2499| 10.936097|2013-09-01 17:43:20|
|  22304233|2013-09-01 01:45:20|       11.923|           1|      2|           0|       0|   1|7.339263| 4.5548677|2013-09-01 01:16:20|
|  51860382|2013-09-01 06:41:20|        3.265|         

In [17]:
peekQuery.lastProgress

res9: org.apache.spark.sql.streaming.StreamingQueryProgress = null


In [18]:
peekQuery.status

res10: org.apache.spark.sql.streaming.StreamingQueryStatus =
{
  "message" : "Processing new data",
  "isDataAvailable" : true,
  "isTriggerActive" : true
}


In [19]:
// peekQuery.stop()

## Write to BigQuery

Write to BigQuery once a minute. BigQuery inserts work better with mini-batched data and this ensures that we give enough time for `alert_1_stats` to accumulate. The [BigQuery connector](https://github.com/GoogleCloudDataproc/spark-bigquery-connector) is installed during cluster setup and loaded automatically when spark-shell session is initiated.

In [31]:
val ingestQuery = anomaly
    .writeStream
    .trigger(Trigger.ProcessingTime(outputTriggerTime))
    .foreachBatch{ 
        (batchDF: DataFrame, batchId: Long) =>
            batchDF.write.format("bigquery")
                .option("table", bigQueryTargetTable)
                .option("temporaryGcsBucket", bigQueryTempBucket)
                .mode(SaveMode.Append)
                .save()
    }.start()

ingestQuery: org.apache.spark.sql.streaming.StreamingQuery = org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@26be343d


In [37]:
ingestQuery.lastProgress

res27: org.apache.spark.sql.streaming.StreamingQueryProgress =
{
  "id" : "4351c9e3-065b-4b26-a830-f6e802042c9e",
  "runId" : "e4a48e28-09a3-4eb7-9f4b-1481d032bc77",
  "name" : null,
  "timestamp" : "2020-03-02T15:32:11.026Z",
  "batchId" : 0,
  "numInputRows" : 66,
  "processedRowsPerSecond" : 0.7863977027654985,
  "durationMs" : {
    "addBatch" : 81093,
    "getBatch" : 0,
    "getEndOffset" : 1,
    "queryPlanning" : 1542,
    "setOffsetRange" : 697,
    "triggerExecution" : 83927,
    "walCommit" : 102
  },
  "eventTime" : {
    "avg" : "2013-09-01T19:03:03.428Z",
    "max" : "2013-09-01T19:19:20.000Z",
    "min" : "2013-09-01T18:47:20.000Z",
    "watermark" : "1970-01-01T00:00:00.000Z"
  },
  "stateOperators" : [ {
    "numRowsTotal" : 35,
    "numRowsUpdated" : 35,
    "memoryUse...

In [35]:
ingestQuery.status

res25: org.apache.spark.sql.streaming.StreamingQueryStatus =
{
  "message" : "Processing new data",
  "isDataAvailable" : true,
  "isTriggerActive" : true
}


In [23]:
//ingestQuery.stop()